Importation du dataset de tweets dans un DataFrame.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

dataset = '/content/drive/MyDrive/WCS/Semaine_13/train.csv'
df_tweet = pd.read_csv(dataset)
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [3]:
df_tweet['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [4]:
df_tweet.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


On ne conserve que les tweets positifs


In [5]:
mask_pos = df_tweet['sentiment'] == 'positive'
mask_neg = df_tweet['sentiment'] == 'negative'

#df_tweet2 ne contient que les tweets positifs
df_tweet2 = df_tweet[mask_pos | mask_neg]

In [6]:
value_count = df_tweet2['sentiment'].value_counts()

In [7]:
nbr_pos = value_count[0]
nbr_neg = value_count[1]
total = nbr_pos + nbr_neg

print(f'il y a {round((nbr_pos/total)*100, 2)}% de tweets positifs, et donc {round((nbr_neg/total)*100, 2)}% de tweets négatifs')

il y a 52.45% de tweets positifs, et donc 47.55% de tweets négatifs


On récupère la liste des stopwords anglais depuis NLTK

In [ ]:
import nltk
nltk.download('popular')

In [9]:
stopwordsenglish = nltk.corpus.stopwords.words("english")
len(stopwordsenglish)

179

On crée une fonction lemma afin de l'appliquer (via la méthode .apply) à toutes les lignes de notre dataframe

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemma(text):
  text_tokens = nlp(text)
  token_lemma = [token.lemma_ for token in text_tokens]
  str_lemma = ' '.join(token_lemma)
  return str_lemma

On applique donc la fonction lemma à la colonne text du DataFrame. On Stocke le résultat dans une nouvelle colonne "lemma" du DataFrame

In [11]:
df_tweet2['lemma'] = df_tweet2['text'].apply(lemma)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Pour crée notre modèle, on transforme les colonnes du dataframe en list, puis un fait notre séparation données tests/données entrainements

In [12]:
X = df_tweet2['lemma'].to_list()
y = df_tweet2['sentiment'].to_list()

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 32)

In [14]:
#On vectorise nos données en enlevant les stopwords
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = stopwordsenglish)
X_train_CV = vectorizer.fit_transform(X_train)
X_train_CV

<12272x13712 sparse matrix of type '<class 'numpy.int64'>'
	with 95192 stored elements in Compressed Sparse Row format>

In [15]:
X_test_CV = vectorizer.transform(X_test)
X_test_CV

<4091x13712 sparse matrix of type '<class 'numpy.int64'>'
	with 28842 stored elements in Compressed Sparse Row format>

In [16]:
#Création et entrainement du modèle
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train_CV,y_train)

#On regarde les scores
print("score de précision sur l'ensemble d'entraînement :",model.score(X_train_CV, y_train))
print("score de précision sur l'ensemble de test :",model.score(X_test_CV, y_test))

score de précision sur l'ensemble d'entraînement : 0.9539602346805737
score de précision sur l'ensemble de test : 0.8636030310437546


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
